# 14.4 团队 AI 工具链搭建

团队 AI 能力落地指南。

**预估成本**: $0（无API调用）

## 1. 工具选型矩阵

| 工具 | 适用场景 | 成本 | 推荐 |
|------|---------|------|------|
| Cursor | 主力开发 | $20/月/人 | ⭐⭐⭐⭐⭐ |
| ChatGPT Plus | 通用问答 | $20/月/人 | ⭐⭐⭐⭐⭐ |
| OpenRouter | 统一API | 按量 | ⭐⭐⭐⭐⭐ |

## 2. AGENTS.md 模板

```markdown
# 项目 AI 开发指南

## 技术栈
- Python 3.11 + FastAPI
- PostgreSQL, Redis

## 代码规范
- 格式化：Black
- 类型注解：必须
- 文档字符串：Google 风格

## AI 使用规则
1. 必须包含类型注解
2. 必须包含错误处理
3. 禁止硬编码敏感信息

## Prompt 模板

### 生成 API
```
生成 FastAPI 端点：POST /api/users
要求：完整类型注解、错误处理、单元测试
```
```

In [ ]:
# 统一 API 配置（OpenRouter）

from openai import OpenAI

def get_ai_client(model='gpt-4o-mini'):
    return OpenAI(
        base_url='https://openrouter.ai/api/v1',
        api_key='sk-or-...'  # 团队共享Key
    )

# 使用
client = get_ai_client()
response = client.chat.completions.create(
    model='openai/gpt-4o-mini',  # 或 anthropic/claude-3.5-sonnet
    messages=[{'role': 'user', 'content': 'Hello'}]
)
print(response.choices[0].message.content)

In [ ]:
# Prompt 模板库

PROMPTS = {
    'generate_api': '''
生成 FastAPI 端点：{method} {path}
功能：{feature}
要求：
- 完整类型注解
- 错误处理
- 文档字符串
- 遵循 AGENTS.md
''',
    'code_review': '''
审查代码，检查：
1. AGENTS.md 规范
2. 类型注解
3. 安全问题

代码：
```
{code}
```
'''
}

def get_prompt(name, **kwargs):
    return PROMPTS[name].format(**kwargs)

# 使用
prompt = get_prompt(
    'generate_api',
    method='POST',
    path='/api/users',
    feature='用户创建'
)
print(prompt)

In [ ]:
# 成本监控

import json
from datetime import datetime
from collections import defaultdict

class AIUsageMonitor:
    def __init__(self, log_file='ai_usage.jsonl'):
        self.log_file = log_file
    
    def log(self, user, model, tokens, cost):
        with open(self.log_file, 'a') as f:
            f.write(json.dumps({
                'timestamp': datetime.now().isoformat(),
                'user': user,
                'model': model,
                'tokens': tokens,
                'cost': cost
            }) + '\n')
    
    def daily_report(self, date=None):
        if not date:
            date = datetime.now().strftime('%Y-%m-%d')
        
        with open(self.log_file, 'r') as f:
            logs = [json.loads(line) for line in f if date in line]
        
        total_cost = sum(log['cost'] for log in logs)
        user_stats = defaultdict(lambda: {'tokens': 0, 'cost': 0})
        
        for log in logs:
            user_stats[log['user']]['tokens'] += log['tokens']
            user_stats[log['user']]['cost'] += log['cost']
        
        alerts = []
        if total_cost > 100:
            alerts.append(f'⚠️  每日成本 ${total_cost:.2f} 超预算')
        
        for user, stats in user_stats.items():
            if stats['cost'] > 20:
                alerts.append(f"⚠️  {user} 超配额：${stats['cost']:.2f}")
        
        return {
            'date': date,
            'total_cost': round(total_cost, 2),
            'total_calls': len(logs),
            'user_stats': dict(user_stats),
            'alerts': alerts
        }

# 示例
monitor = AIUsageMonitor()
monitor.log('张三', 'gpt-4o-mini', 500, 0.05)
report = monitor.daily_report()
print(json.dumps(report, indent=2, ensure_ascii=False))

## 3. 培训计划

**第1课：工具安装（1小时）**
- Cursor/Cline 配置
- 第一个AI编码任务

**第2课：实战演练（1周）**
- 完成1个真实需求
- 结对编程

**第3课：最佳实践（持续）**
- 每周技术分享
- 内部案例库

**第4课：高级技巧（按需）**
- MCP Server 开发
- Multi-Agent 应用

## 4. 成本预算

| 项目 | 月预算 | 说明 |
|------|--------|------|
| Cursor | $20×人数 | 编码工具 |
| ChatGPT | $20×骨干 | 问答 |
| API | $200-500 | 自建应用 |
| 储备金 | $100 | 应对超支 |

**小团队（5-10人）**：~$150/月
**中型团队（10-50人）**：~$800/月